# Identifying Potential Carbon Storage Facilities

## Import and Procedural Functions

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import folium
import contextily as cx
import rtree
from zlib import crc32
import hashlib
from shapely.geometry import Point, LineString, Polygon
import numpy as np
from scipy.spatial import cKDTree
from shapely.geometry import Point
from haversine import Unit

## Restrictions

* Must be near a pipeline terminal
* Must be Near a petrolium Port 

### Query Plan

Imports
* Import LNG terminal Dataa
* Import well data

Filtering
* for each well calculate nearest terminal
* for each well calculate distance from nearest terminal
* eliminate wells further than 15 miles from a terminal



## Data Frame Import

### Wells DataFrame

In [45]:
## Importing our DataFrames

gisfilepath = "/Users/jnapolitano/Projects/data/energy/non-active-wells.geojson"


wells_df = gpd.read_file(gisfilepath)

wells_df = wells_df.to_crs(epsg=3857)



### Terminal DataFrame

In [46]:
## Importing our DataFrames

gisfilepath = "/Users/jnapolitano/Projects/data/energy/Liquified_Natural_Gas_Import_Exports_and_Terminals.geojson"


terminal_df = gpd.read_file(gisfilepath)

terminal_df = terminal_df.to_crs(epsg=3857)



### Eliminating SUSPENDED Terminal from the DataFrame

In [41]:
terminal_df.drop(terminal_df[terminal_df['STATUS'] == 'SUSPENDED'].index, inplace = True)
terminal_df.rename(columns={"NAME": "TERMINAL_NAME"})
terminal_df['TERMINAL_GEO'] = terminal_df['geometry'].copy()
terminal_df.columns

Index(['OBJECTID', 'TERMID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4',
       'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY', 'COUNTYFIPS',
       'COUNTRY', 'LATITUDE', 'LONGITUDE', 'NAICS_CODE', 'NAICS_DESC',
       'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'EPA_ID',
       'ALT_NAME', 'OWNER', 'POSREL', 'JRSDCTN', 'CONTYPE', 'IE_PORT',
       'BERTHS', 'STORAGE', 'STORCAP', 'CURRENTCAP', 'APPCAP', 'OPYEAR',
       'IMPEXPCTRY', 'VOLUME', 'PRICE', 'geometry', 'TERMINAL_GEO'],
      dtype='object')

### Plotting Terminal by TYPE

In [42]:
terminal_map =terminal_df.explore(
    column="TYPE", # make choropleth based on "PORT_NAME" column
     popup=True, # show all values in popup (on click)
     tiles="CartoDB positron", # use "CartoDB positron" tiles
     cmap='Set1', # use "Set1" matplotlib colormap
     #style_kwds=dict(color="black"),
     marker_kwds= dict(radius=6),
     #tooltip=['NAICS_DESC','REGION', 'COMMODITY' ],
     legend =True, # use black outline)
     categorical=True,
    )


terminal_map

### Terminal Impressions

According to the data there is not an export nor import location on The Western Side of the United States.

East Asian import or carbon capture export demands could justfity port development.  Another study must be conducted.

## Filtering Wells by Terminal Distance

In [36]:
def ckdnearest(gdA, gdB):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name='dist')
        ], 
        axis=1)

    return gdf

c = ckdnearest(wells_df, terminal_df)
c

,level_0,index,OBJECTID,ID,NAME,STATE,TYPE,STATUS,COUNTY,COUNTYFIPS,...,STORAGE,STORCAP,CURRENTCAP,APPCAP,OPYEAR,IMPEXPCTRY,VOLUME,PRICE,TERMINAL_GEO,dist
0,0,309419,309420,W540019397,NOT AVAILABLE,WV,OIL & NATURAL GAS WELL,NON-ACTIVE WELL,HARRISON,54033,...,7,14.8,1.800,-999.000,1978,"TTO, NGA",9.1,9.37,POINT (-76.40909 38.38960),4.109077
1,1,309427,309428,W540019405,NOT AVAILABLE,WV,OIL & NATURAL GAS WELL,NON-ACTIVE WELL,HARRISON,54033,...,7,14.8,1.800,-999.000,1978,"TTO, NGA",9.1,9.37,POINT (-76.40909 38.38960),4.074361
2,2,309417,309418,W540019395,NOT AVAILABLE,WV,OIL & NATURAL GAS WELL,NON-ACTIVE WELL,HARRISON,54033,...,7,14.8,1.800,-999.000,1978,"TTO, NGA",9.1,9.37,POINT (-76.40909 38.38960),4.121660
3,3,309416,309417,W540019394,NOT AVAILABLE,WV,OIL & NATURAL GAS WELL,NON-ACTIVE WELL,HARRISON,54033,...,7,14.8,1.800,-999.000,1978,"TTO, NGA",9.1,9.37,POINT (-76.40909 38.38960),4.153905
4,4,309418,309419,W540019396,NOT AVAILABLE,WV,OIL & NATURAL GAS WELL,NON-ACTIVE WELL,HARRISON,54033,...,7,14.8,1.800,-999.000,1978,"TTO, NGA",9.1,9.37,POINT (-76.40909 38.38960),4.058587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155202,155202,670441,670442,W210005761,NOT AVAILABLE,KY,OIL & NATURAL GAS WELL,STORAGE WELL/MAINTENANCE WELL/OBSERVATION WELL,PERRY,21193,...,5,11.5,0.035,0.315,1978,NOT APPLICABLE,0.0,0.00,POINT (-80.99712 32.08601),5.527314
155203,155203,670504,670505,W210005824,NOT AVAILABLE,KY,OIL & NATURAL GAS WELL,STORAGE WELL/MAINTENANCE WELL/OBSERVATION WELL,WEBSTER,21233,...,2,7.5,1.500,-999.000,2011,NOT APPLICABLE,0.0,0.00,POINT (-88.50311 30.32248),7.223410
155204,155204,670505,670506,W210005825,NOT AVAILABLE,KY,OIL & NATURAL GAS WELL,STORAGE WELL/MAINTENANCE WELL/OBSERVATION WELL,WEBSTER,21233,...,2,7.5,1.500,-999.000,2011,NOT APPLICABLE,0.0,0.00,POINT (-88.50311 30.32248),7.213539
155205,155205,269536,269537,W490000567,ROCK HOUSE U 30,UT,OIL & NATURAL GAS WELL,STORAGE WELL/MAINTENANCE WELL/OBSERVATION WELL,UINTAH,49047,...,2,-999.0,1.000,-999.000,2019,NOT APPLICABLE,-999.0,-999.00,POINT (-116.84741 31.98852),10.897427


In [33]:
c.describe()

,level_0,index,OBJECTID,LATITUDE,LONGITUDE,PERMITNO,OPERATORID,SURF_LAT,SURF_LONG,BOT_LAT,...,LATITUDE,LONGITUDE,BERTHS,STORAGE,STORCAP,CURRENTCAP,APPCAP,VOLUME,PRICE,dist
count,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,1.552070e+05,1.552070e+05,155207.000000,155207.000000,155207.000000,...,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000
mean,77603.000000,330141.223038,330142.223038,37.388915,-88.688821,2.031513e+06,6.847616e+07,-66.440744,-179.323395,-982.734015,...,33.823444,-87.324160,-41.416244,-38.553100,-136.088432,1.519527,-865.252498,-142.223472,-142.373735,6.019991
std,44804.545952,232346.189714,232346.189714,4.392454,8.573390,3.251915e+06,2.567834e+08,311.099627,273.538141,129.138204,...,3.832021,12.344338,203.633934,204.252166,357.363524,0.655252,340.434877,355.031345,354.775699,4.459349
min,0.000000,0.000000,1.000000,28.899560,-123.342380,-9.990000e+02,-9.990000e+02,-999.000000,-999.000000,-999.000000,...,27.889869,-116.847415,-999.000000,-999.000000,-999.000000,0.000000,-999.000000,-999.000000,-999.000000,0.004124
25%,38801.500000,134028.500000,134029.500000,32.871595,-93.874935,-9.990000e+02,-9.990000e+02,32.028420,-93.939920,-999.000000,...,30.112960,-93.288224,2.000000,2.000000,7.500000,1.500000,-999.000000,0.000000,0.000000,2.873130
50%,77603.000000,310196.000000,310197.000000,38.396300,-87.754143,-9.990000e+02,-9.990000e+02,38.189080,-88.392278,-999.000000,...,31.988522,-88.503111,2.000000,4.000000,11.000000,1.800000,-999.000000,0.000000,0.000000,4.802812
75%,116404.500000,348997.500000,348998.500000,39.222220,-81.201900,3.502644e+06,-9.990000e+02,39.205850,-81.203860,-999.000000,...,38.389603,-76.409092,2.000000,7.000000,14.800000,1.800000,-999.000000,9.100000,9.370000,7.515025
max,155206.000000,969099.000000,969100.000000,50.744220,-75.593800,2.017004e+07,1.044775e+09,50.744220,-75.593800,45.179110,...,38.389729,-76.408824,2.000000,7.000000,18.000000,4.000000,4.000000,932.000000,11.340000,22.197154


In [50]:
nearest_wells_df= wells_df.sjoin_nearest(terminal_df, distance_col="distances")

In [51]:
nearest_wells_df.describe()

,level_0,index,OBJECTID_left,LATITUDE_left,LONGITUDE_left,PERMITNO,OPERATORID,SURF_LAT,SURF_LONG,BOT_LAT,...,LATITUDE_right,LONGITUDE_right,BERTHS,STORAGE,STORCAP,CURRENTCAP,APPCAP,VOLUME,PRICE,distances
count,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,1.552070e+05,1.552070e+05,155207.000000,155207.000000,155207.000000,...,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,155207.000000,1.552070e+05
mean,77603.000000,330141.223038,330142.223038,37.388915,-88.688821,2.031513e+06,6.847616e+07,-66.440744,-179.323395,-982.734015,...,34.272492,-86.967254,-33.938179,-30.905661,-129.092163,1.649174,-937.566498,-134.892939,-135.078039,7.509743e+05
std,44804.545952,232346.189714,232346.189714,4.392454,8.573390,3.251915e+06,2.567834e+08,311.099627,273.538141,129.138204,...,3.939841,12.573387,185.950962,186.546056,350.481192,0.519726,240.231025,348.246757,348.084448,6.088721e+05
min,0.000000,0.000000,1.000000,28.899560,-123.342380,-9.990000e+02,-9.990000e+02,-999.000000,-999.000000,-999.000000,...,27.889869,-116.847415,-999.000000,-999.000000,-999.000000,0.000000,-999.000000,-999.000000,-999.000000,5.272348e+02
25%,38801.500000,134028.500000,134029.500000,32.871595,-93.874935,-9.990000e+02,-9.990000e+02,32.028420,-93.939920,-999.000000,...,30.112960,-93.288224,2.000000,2.000000,7.500000,1.500000,-999.000000,0.000000,0.000000,3.683350e+05
50%,77603.000000,310196.000000,310197.000000,38.396300,-87.754143,-9.990000e+02,-9.990000e+02,38.189080,-88.392278,-999.000000,...,31.988522,-88.503111,2.000000,4.000000,11.000000,1.800000,-999.000000,0.000000,0.000000,5.383780e+05
75%,116404.500000,348997.500000,348998.500000,39.222220,-81.201900,3.502644e+06,-9.990000e+02,39.205850,-81.203860,-999.000000,...,38.389603,-76.409092,2.000000,7.000000,14.800000,1.800000,-999.000000,9.100000,9.370000,9.919574e+05
max,155206.000000,969099.000000,969100.000000,50.744220,-75.593800,2.017004e+07,1.044775e+09,50.744220,-75.593800,45.179110,...,42.392856,-71.058151,2.000000,7.000000,18.000000,4.000000,4.000000,932.000000,11.340000,3.191132e+06


In [ ]:
df.geopy.distance.distance(coords_1, coords_2).km

In [ ]:
#### Take the the dataframe
##### For each row basically compare geo to port geo
##### distance = what return

nearest_wells_df['test'] = nearest_wells_df.apply(lambda x: print(x), axis=1)


